# Tarefa Aula 6 Engenharia de Dados Awari
## Rogerio Veiga Andrade - 07/11/2023

### Criar script para ler os datasets em /exercicios/municipios-estados — o script deve:
#### criar uma pasta nomeada com a sigla da UF para cada estado encontrado no arquivo JSON;
#### organizar as cidades por estado um único arquivo CSV, nomeado como cidades.csv;
#### salvar esse arquivo cidades.csv dentro da pasta da UF (estado) correspondente.
### Importar as pastas e arquivos salvos para um bucket no MinIO.
### Exportar os dados para o MongoDB e visualizá-los a partir do banco de dados.

In [24]:
# Importando bibliotecas
import numpy as np
import pandas as pd
from io import BytesIO
from pymongo import MongoClient
import boto3
from io import StringIO 

In [25]:
# Criando o objeto client

client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='6DdjRBILhyMi2G2i',
    aws_secret_access_key='9LSWJZT7f0i8FgjzflQQYrOq9tvUQrD1',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [26]:
# Importando e tratando os estados

estados = pd.read_json('arquivos/estados.json')
estados = estados.rename(columns = {'nome': 'nome do estado', 'latitude': 'latitude do estado', 'longitude': 'longitude do estado'})
estados.head()

,codigo_uf,uf,nome do estado,latitude do estado,longitude do estado,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte


In [27]:
# Importando os municípios

municipios = pd.read_csv('arquivos/municipios.csv')
municipios.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [28]:
# Juntando os dataframes para ter o estado de cada município no mesmo dataframe

municipios_estados = municipios.merge(estados, on='codigo_uf', how='inner')
municipios_estados.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf,nome do estado,latitude do estado,longitude do estado,regiao
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52,1050,62,America/Sao_Paulo,GO,Goiás,-15.98,-49.86,Centro-Oeste
1,5200100,Abadiânia,-16.1970,-48.7057,0,52,9201,62,America/Sao_Paulo,GO,Goiás,-15.98,-49.86,Centro-Oeste
2,5200134,Acreúna,-17.3960,-50.3749,0,52,9645,64,America/Sao_Paulo,GO,Goiás,-15.98,-49.86,Centro-Oeste
3,5200159,Adelândia,-16.4127,-50.1657,0,52,9769,64,America/Sao_Paulo,GO,Goiás,-15.98,-49.86,Centro-Oeste
4,5200175,Água Fria de Goiás,-14.9778,-47.7823,0,52,9771,62,America/Sao_Paulo,GO,Goiás,-15.98,-49.86,Centro-Oeste


In [29]:
# Enviando para o MinIO um arquivo chamado cidades.csv com as cidades para a pasta de cada estado

for uf in estados['uf'].unique():
    csv_buffer = StringIO()
    df_tmp = municipios_estados.loc[ municipios_estados['uf'] == uf ]
    df_tmp.to_csv(csv_buffer)
    client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-06', Key=f'{uf}/cidades.csv')
    

In [41]:
# Teste de leitura com cidades da Bahia

df = pd.read_csv(client.get_object(Bucket='aula-06', Key='BA/cidades.csv')['Body'])
df.head()

,Unnamed: 0,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf,nome do estado,latitude do estado,longitude do estado,regiao
0,1427,2900108,Abaíra,-13.24880,-41.6619,0,29,3301,77,America/Sao_Paulo,BA,Bahia,-13.29,-41.71,Nordeste
1,1428,2900207,Abaré,-8.72073,-39.1162,0,29,3303,75,America/Sao_Paulo,BA,Bahia,-13.29,-41.71,Nordeste
2,1429,2900306,Acajutiba,-11.65750,-38.0197,0,29,3305,75,America/Sao_Paulo,BA,Bahia,-13.29,-41.71,Nordeste
3,1430,2900355,Adustina,-10.54370,-38.1113,0,29,3253,75,America/Sao_Paulo,BA,Bahia,-13.29,-41.71,Nordeste
4,1431,2900405,Água Fria,-11.86180,-38.7639,0,29,3307,75,America/Sao_Paulo,BA,Bahia,-13.29,-41.71,Nordeste


In [61]:
# Lista dos prefixos (pastas) no bucket

response = client.list_objects_v2(Bucket='aula-06', Delimiter='/')
response['CommonPrefixes']

[{'Prefix': 'AC/'},
 {'Prefix': 'AL/'},
 {'Prefix': 'AM/'},
 {'Prefix': 'AP/'},
 {'Prefix': 'BA/'},
 {'Prefix': 'CE/'},
 {'Prefix': 'DF/'},
 {'Prefix': 'ES/'},
 {'Prefix': 'GO/'},
 {'Prefix': 'MA/'},
 {'Prefix': 'MG/'},
 {'Prefix': 'MS/'},
 {'Prefix': 'MT/'},
 {'Prefix': 'PA/'},
 {'Prefix': 'PB/'},
 {'Prefix': 'PE/'},
 {'Prefix': 'PI/'},
 {'Prefix': 'PR/'},
 {'Prefix': 'RJ/'},
 {'Prefix': 'RN/'},
 {'Prefix': 'RO/'},
 {'Prefix': 'RR/'},
 {'Prefix': 'RS/'},
 {'Prefix': 'SC/'},
 {'Prefix': 'SE/'},
 {'Prefix': 'SP/'},
 {'Prefix': 'TO/'},
 {'Prefix': 'exemplos/'}]